In [1]:
!pip install git+https://github.com/afewell/agency-swarm.git@notebook-update-afewell gradio duckduckgo-search

  Cloning https://github.com/afewell/agency-swarm.git (to revision notebook-update-afewell) to /tmp/pip-req-build-gqo7fhvz
  Running command git clone --filter=blob:none --quiet https://github.com/afewell/agency-swarm.git /tmp/pip-req-build-gqo7fhvz
  Running command git checkout -b notebook-update-afewell --track origin/notebook-update-afewell
  Switched to a new branch 'notebook-update-afewell'
  Branch 'notebook-update-afewell' set up to track remote branch 'notebook-update-afewell' from 'origin'.
  Resolved https://github.com/afewell/agency-swarm.git to commit 763ec3526b018a6ebb3f2b9e683d58627819ce33
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from agency_swarm import set_openai_key
from getpass import getpass
set_openai_key(getpass("Please enter your openai key: "))

Please enter your openai key:  ········


# Project Manager Agent


In [3]:
ea_instructions = """Instructions for Project Manager
As the Project Manager with editorial expertise, you represent the apex of project management fused with editorial acumen, acting as the cornerstone for high-profile blogging projects for top-tier organizations. Your communication with both clients and team members should reflect not only authoritative project management but also an astute editorial insight. Follow these slightly updated instructions to enhance your dual role:

1. Project Initiation and Planning
Client Consultation: Engage deeply with the client to grasp their vision, needs, and editorial expectations. Utilize your dual expertise to ask probing questions that clarify both project scope and content direction.

Project Plan Development: Craft a detailed project plan that encompasses step-by-step tasks, including specific editorial milestones, timelines, resource allocation, and quality benchmarks. Employ project management and editorial tools to visualize the roadmap and ensure excellence at every stage.

Task Delegation with Editorial Perspective: When assigning tasks, consider both the agent's project-related skills and their understanding of content quality. Match tasks to agents’ strengths to optimize project outcomes and content excellence.

2. Execution and Monitoring
Objective Clarification with Editorial Focus: Clearly define each task's objectives, including editorial goals and quality expectations, to ensure content aligns with client aspirations and our agency standards.

Facilitate Expert Collaboration: Foster a collaborative environment where agents can share progress and insights, ensuring editorial considerations are integrated into every phase of content creation.

Quality Assurance and Editorial Review: Rigorously evaluate completed tasks for both project alignment and editorial quality, ensuring the content meets our high standards of clarity, insight, and engagement.

3. Adaptation and Improvement
Iterative Feedback and Editorial Guidance: Provide targeted feedback focused on both project improvement and content refinement. Encourage agents to enhance their contributions through an iterative process, emphasizing editorial excellence.

Adaptive Project and Content Management: Be ready to adjust the project and editorial strategy in response to new insights, client feedback, or evolving content standards.

4. Communication and Reporting
Ongoing Communication with an Editorial Lens: Keep all stakeholders informed with updates that cover project progress and content development, addressing any concerns with a proactive, solution-oriented approach.

Comprehensive Reporting: Compile a detailed report upon project completion, summarizing both project outcomes and content achievements, with recommendations for future editorial and project enhancements.

5. Reflective Practice
Post-Project Review with an Editorial Focus: Conduct a thorough review to evaluate project and content success, integrating feedback from clients and team members to inform future strategies.

Professional Development in Project and Editorial Management: Continuously seek to improve your expertise in project management and editorial practices, keeping pace with industry innovations that could elevate project and content outcomes.

This role is pivotal to the success of our projects, blending precise project management with editorial finesse to ensure each project serves as a benchmark of excellence in both management and content quality."""

In [4]:
from agency_swarm import Agent

ea = Agent(name="Project Manager",
            description="Responsible for overseeing the entire blogging project, ensuring quality control, and managing client communications.",
            instructions=ea_instructions, # can be a file like ./instructions.md
            files_folder=None,
            tools=[])

# Virtual Assistant

### Custom tools

In [5]:
from duckduckgo_search import DDGS
from pydantic import Field
from agency_swarm.util.oai import get_openai_client
from agency_swarm import BaseTool


client = get_openai_client()


class SearchWeb(BaseTool):
    """Search the web with a search phrase and return the results."""

    phrase: str = Field(..., description="The search phrase you want to use. Optimize the search phrase for an internet search engine.")

    # This code will be executed if the agent calls this tool
    def run(self):
      with DDGS() as ddgs:
        return str([r for r in ddgs.text(self.phrase, max_results=3)])


In [6]:
ra_instructions = """Instructions for the Researcher
Your role is to dive deep into the fabric of the topic, uncovering insights that provide our blogs with the unparalleled depth and perspective our clients expect.

Comprehensive Research: Conduct extensive web searches on the given topics, employing advanced search techniques to uncover unique data, studies, and perspectives that are not readily accessible.

Continuous Exploration: Be prepared to conduct additional research during the revision phases to enhance the blog's content with new insights or to address any gaps identified by the Project Manager or Blogger.

Synthesize Findings: Provide a synthesized summary of your research findings, highlighting the most critical insights that will add significant value to the blog.

Cite Sources: Ensure all data and insights are accurately cited, maintaining the integrity and credibility of our content.

Do not communicate directly with the Blogger Agent, all communications should go through the Project Manager.
"""

In [7]:
ra = Agent(name="Research Agent",
            description="Responsible for conducting in-depth research to support blog development.",
            instructions=ra_instructions,
            files_folder=None,
            tools=[SearchWeb])

# Blogger Agent

In [8]:
ba_instructions = """Instructions for the Blogger
As the voice of our agency, your writing must captivate, inform, and inspire our discerning audience.

Engaging Content Creation: Write blogs that not only adhere to the highest standards of journalistic and literary quality but also engage and enlighten our sophisticated audience.

Adherence to Research: Integrate the research findings seamlessly into your writing, ensuring the blog is informative, insightful, and adds significant value to the discourse on the topic.

Revision and Refinement: Work closely with the Project Manager, incorporating feedback meticulously to refine and elevate the blog to meet our world-class standards.

Originality and Creativity: Ensure each blog is a testament to original thought, presenting unique perspectives and mature insights that reflect the caliber of publications like The Atlantic or The New York Times.

Do not communicate directly with the research agent, all communications including research requests should go through the Project Manager.
"""

In [9]:
ba = Agent(name="Blogger Agent",
            description="Responsible for writing blog posts and incorporating research findings into compelling narratives.",
            instructions=ba_instructions, # can be a file like ./instructions.md
            files_folder=None,
            tools=[])

# Create Agency

In [10]:
agency_manifesto = """# "Elev8 Media" Agency Manifesto

You are a part of a world class media agency called "Elev8 Media"

The agency manifesto is "Elevating Discourse, One Blog at a Time"
"""

In [11]:
from agency_swarm import Agency

agency = Agency([
    ea,
    [ea, ra],
    [ea, ba],
    [ra, ba]
], shared_instructions=agency_manifesto)

Updating assistant... Blogger Agent


In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

# Demo with Gradio

In [ ]:
agency.run_demo()

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

USER:  Please Write a blog about the latest advancements in VR and AR, and how they're transforming industries from gaming to education. Highlight some of the most exciting VR/AR experiences currently available


THREAD:[ user -> Project Manager ]: URL https://platform.openai.com/playground?assistant=asst_t4QV6KSvgwzW7ykcoO3SGq9d&mode=assistant&thread=thread_6XUs9QhHTdlsKbRC8aHwgzSO


───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

👤User 🗣️ @Project Manager

...

🐻SearchWeb ⚙️Function Output

[{'title': '10 most innovative companies in augmented and virtual reality of 2023', 'href': 
'https://www.fastcompany.com/90846787/most-innovative-companies-augmented-virtual-reality-2023', 'body': 'The 10 
most innovative companies in augmented and virtual reality of 2023, including Unity, Meta, Google, Niantic, and 
Varjo.'}, {'title': "How Unity earned the title of world's most innovative AR/VR company", 'href': 
'https://blog.unity.com/news/unity-named-2023-most-innovative-ar-vr-company', 'body': "Unity has been named to Fast
Company 's prestigious annual list of the World's Most Innovative Companies for 2023, which highlights businesses 
at the forefront of their respective industries."}, {'title': 'The 10 Biggest AR, VR And Immersive Internet Trends 
In 2024 - Forbes', 'href': 
'https://www.forbes.com/sites/bernardmarr/2023/10/13/the-10-biggest-ar-vr-and-immersive-internet-trends-in-2024/', 
'body': 'The biggest technology sensation of 2023 (and no doubt beyond) has many applications that align with the 
immersive, engaging online experiences that will be created in 2024. An obvious use case...'}]

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

🐻Research Agent 🗣️ @Project Manager

The latest advancements in VR (Virtual Reality) and AR (Augmented Reality) technology have seen significant 
announcements and developments:

1. **CES 2023 Highlights** - CES 2023 showcased several new technologies and products, including the PlayStation 
VR2, a new HTC VIVE headset, and developments from Lumus in AR. ([Tom's 
Guide](https://www.tomsguide.com/features/all-the-biggest-vr-and-ar-announcements-at-ces-2023))

2. **Emerging Trends for 2023/2024** - The VR and AR fields are moving towards more immersive Internet trends, 
including the anticipation of what 2024 will bring with AI integration and immersive online experiences. Bernard 
Marr at Forbes also talks about the biggest trends and applications in these technologies. 
([Forbes](https://www.forbes.com/sites/bernardmarr/2023/10/13/the-10-biggest-ar-vr-and-immersive-internet-trends-in
-2024/), [Cable Matters](https://www.cablematters.com/Blog/Virtual-Reality/2023-technology-trends))

When it comes to the **applications of VR and AR across different industries**:

1. **Virtual Reality Applications** - VR is being used across 10 major industries, impacting everything from design
to training processes. ([Trio](https://www.trio.dev/blog/virtual-reality-applications))

2. **Business Applications** - Insight.com highlights how AR and VR will play increasingly important roles in 
retail and various business sectors, forecasting the market to reach nearly $300 billion by 2024. 
([Insight](https://www.insight.com/en_US/content-and-resources/2023/ar-and-vr-business-applications-in-2023-and-bey
ond.html))

3. **Market Growth** - The AR/VR industries are projected to grow from $13.8 billion in revenues in 2022 to $50.9 
billion in 2026, stressing the significant role this technology will have in many industries. ([Radiant Vision 
Systems](https://www.radiantvisionsystems.com/blog/future-looks-bright-ar/vr/mr-2023-beyond))

Regarding some of the **most innovative VR/AR experiences currently available**:

1. **Innovative Companies** - Fast Company lists the most innovative companies in AR and VR for 2023, including 
Unity, Meta, Google, Niantic, and Varjo. ([Fast 
Company](https://www.fastcompany.com/90846787/most-innovative-companies-augmented-virtual-reality-2023))

2. **Unity's Rise** - Unity has been recognized by Fast Company as the world's most innovative AR/VR company for 
2023, emphasizing their contribution to the industry. ([Unity 
Blog](https://blog.unity.com/news/unity-named-2023-most-innovative-ar-vr-company))

3. **Immersive Trends** - Forbes discusses how the technology sensation of 2023 will continue to provide immersive 
and engaging online experiences into 2024 and beyond. 
([Forbes](https://www.forbes.com/sites/bernardmarr/2023/10/13/the-10-biggest-ar-vr-and-immersive-internet-trends-in
-2024/))

If more detailed information or a deeper dive into these topics is required, I can continue to conduct further 
research to supplement these findings.